# MLE estimation for Poisson process demand measured over Voronoï cells

## Poisson demand model

Assume that in a region $X\subset \mathbb{R}^2$ we have an independently marked Poisson process with spatial intensity $\lambda(x)$ and mark distribution $F(dy)$ with density $f(y)$ concentrated on the positive real line,

Let $\{s_i\}_{i=1,\ldots,s} \subset X$ be a set of sites and $V_i$ the corresponding Voronoï cells induced by the sites, i.e. 

$$V_i = \{x\in X: d(x,s_i)\leqslant d(x,s_j) \; \forall j = 1,\ldots,s\}.$$

Denote the Poisson process by:
$$\Phi = \sum_n \delta_{(x_n,\sigma_n)},$$
where $x_n$ are the point locations and $\sigma_n$ the mark with distribution $F$.

Consider the random variables:
$$Y_i = \sum_{n} \sigma_n \mathbf{1}_{\{x_n\in V_i\}} = \int \sigma \mathbf{1}_{V_i}(x) d\Phi$$
We interpret $Y_i$ as the total demand in cell $V_i$ measured at the point $s_i$, where each demand is realized at a point $x_n$ and value $\sigma_n$. We denote by $N_i$ the random count of points in cell $i$.

Since the Voronoï cells are disjoint by construction, the random variables $Y_i$ are independent by the independence property of the underlying  Poisson process.

## Parametric model

We now introduce a parametric model for demand. Specifically, we assume that $\lambda(x;\theta)$ comes from a mixture of RBF functions, i.e.:

$$\lambda(x;\theta) = \sum_{j=1}^k w_j e^{-\frac{1}{2\sigma_j^2}(x-\mu_j)^T(x-\mu_j)}$$

As for the density $f$, we choose an exponential distribution with parameter $\nu$. Let $\theta$ include all the aforementioned parameters, i.e. $w_j$, $\mu_j$, $\sigma_j$ for $j=1,\ldots,k$ and $\nu$. We would like to perform a Maximum Likelihood Estimation of $\theta$ from a realization of the variables $Y_i=y_i$, $i=1,\ldots, s$.

Since $Y_i$ are independent random variables, we compute the likelihood for fixed $i$ first.

$$L_i(\theta; y_i) = \sum_{n=0}^\infty P(N_i=n) f^{(*n)}(y_i).$$

where $f^{(*n)}$ denotes the $n$ convolution of the density $f$, with $f^{(*0)}=\delta_0$. In our case, where $f$ is exponential, this corresponds to the Erlang distribution with paramter $n$:

$$f^{(*n)}(y) = \frac{1}{(n-1)!}\nu^n y^{n-1}e^{-\nu y}.$$

The complete likelihood is thus given by:

$$L_i(\theta; y_i) = \sum_{n=0}^\infty e^{-\lambda_i(\theta)} \frac{(\lambda_i(\theta))^n}{n!} \frac{1}{(n-1)!}\nu^n y_i^{n-1}e^{-\nu y_i},$$

where we have denoted $\lambda_i(\theta) = \int_{V_i} \lambda(x;\theta) dx$.

Maximizing this likelihood function becomes intractable. The main issue comes from the fact that the Poisson counts $N_i$ cannot be observed. The problem may be treated using the well known Expectation-Maximization algorithm, but first we will pursue another approach, based on the Viterbi algorithm for Hidden Markov Models.

## Maximizing the combined likelihood.

Instead of working with $L_i(\theta,y_i)$, let us instead work with the joint likelihood $L_i(\theta,n_i,y_i)$ where $n_i$ are the (unobserved) Poisson counts on cell $i$. In that case, we can compute the log-likelihood as:

$$\ell_i(\theta,n_i,y_i) := \log L_i(\theta;n_i,y_i) = -\lambda_i(\theta) + n_i\log(\lambda_i(\theta)) + n_i\log(\nu) + (n_i-1)\log(y_i) - \nu y_i - \log(n_i!) - \log((n_i-1)!)$$

Starting for an initial guess of the parameters $\theta_0$, we can first maximize the likelihood of the unobserved counts $n_i$ maximizing over this variable. Then, given the counts $n_i$, we can maximize:
$$\ell(\theta;n,y) = \sum_{i=1}^s \ell_i(\theta,n_i,y_i).$$
Iterating over this process, one obtains a sequence of estimates $\theta^{(k)}$ with increasing log-likelihood. 

### Obtaining the counts

To obtain the counts given $\theta$ and $y_i$, observe first that if $y_i=0$, then $n_i=0$ is the only point with positive probability and thus we choose $n_i=0$ as the estimate. If $y_i>0$, then $n_i=0$ has null probability son we maximize over $n\geqslant 1$. Define, for $n\geqslant 1$:

$$a_n := L_i(\theta;n,y_i) = e^{-\lambda_i(\theta)} \frac{(\lambda_i(\theta))^n}{n!} \frac{1}{(n-1)!}\nu^n y_i^{n-1}e^{-\nu y_i}.$$

And compute:

$$\frac{a_{n+1}}{a_n} = \frac{\lambda_i(\theta)\nu y_i}{(n+1)n}$$

Note that the ratio is greater that $1$ iff:

$$n(n+1)\leqslant \lambda_i(\theta)\nu y_i.$$

Since $n(n+1)$ is increasing in $n\geqslant 1$, $a_n$ is maximized at:

$$n_i = \left\lfloor \frac{-1+\sqrt{1+4\lambda_i(\theta)\nu y_i}}{2}\right\rfloor$$

We can interpret the above estimate in the case the counts are large, i.e. $\lambda_i(\theta)\nu y_i \gg 1$. In that case, the estimate becomes:

$$n_i = \lfloor \sqrt{\lambda_i(\theta)\nu y_i}\rfloor$$

Note that for a given choice of the parameters, we have two reasonable estimations of $n_i$. One is simply the expected number of points $\lambda_i(\theta)$. The other is, given that we observe $y_i$ and each point contributes $1/\nu$ on average, the expected number of points is $\nu y_i$. The most likely estimator in this case is in fact the geometric mean of both estimates.

### Maximizing the likelihood

Once we have the estimate of the unobserved variables, we can write the joint likelihood as a function of $\theta$ as:

$$\ell(\theta,n,y) = \sum_i \ell_i(\theta,n_i,y_i) = \sum_i -\lambda_i(\theta) + n_i\log(\lambda_i(\theta)) + n_i\log(\nu) + (n_i-1)\log(y_i) - \nu y_i - \log(n_i!) - \log((n_i-1)!)$$

The new estimate for $\nu$ follows immediately by differentiation:

$$0=\frac{\partial \ell}{\partial \nu} = \frac{1}{\nu} \sum_i n_i - \sum_i y_i$$

and thus:

$$\hat{\nu} = \frac{\sum_i n_i}{\sum_i y_i}$$

i.e. the inverse of the average demand for the given counts, which is expected due to the exponential distribution chosen.

As for the maximization of the remaining parameters, we resort to a gradient step, since the derivatives $\frac{\partial \lambda_i(\theta)}{\partial \theta}$ can be computed for the case of the chosen RBF family as we did in the previous case. Differentiating $\ell$ we get:

$$\frac{\partial \ell}{\partial \theta} = \sum_i \frac{\partial \ell}{\partial \lambda_i}\frac{\partial \lambda_i}{\partial \theta} = \sum_i \left[-1 + \frac{1}{\lambda_i(\theta)}\right]\frac{\partial \lambda_i(\theta)}{\partial \theta}.$$

Note that only one gradient step can be performed since that will get an increasing sequence.